In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset=pd.read_csv('/content/2014_ebola_CF_labeled_data.tsv',delimiter='\t',quoting=3)

In [4]:
dataset.head()

,tweet_id,tweet_text,label
0,'497671828195008512',RT @QuestionsNG: Ebola is not gonna come close...,other_useful_information
1,'522457018863726592',It's too funny to watch MSNBC put an infectiou...,treatment
2,'523158748958031873',RT @mikecalma: How the #GOP fights infectious ...,other_useful_information
3,'522494238815031296',@drbrainbugz it's all about spin! Guaranteed t...,other_useful_information
4,'523510656771698689',WHO officially declares that #Senegal has ende...,disease_transmission


In [5]:
dataset.shape

(1774, 3)

In [9]:
print(set(dataset.label))

{'prevention', 'deaths_reports', 'not_related_or_irrelevant', 'other_useful_information', 'disease_signs_or_symptoms', 'affected_people', 'treatment', 'disease_transmission'}


In [10]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
for i in range(0,1774):
    review=re.sub('[^a-zA-Z]',' ',dataset['tweet_text'][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
len(corpus)

1774

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=8000)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,2].values

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [15]:
y_pred = classifier.predict(X_test)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [17]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, digits=9))

                           precision    recall  f1-score   support

          affected_people  0.350000000 0.179487179 0.237288136        39
           deaths_reports  0.363636364 0.444444444 0.400000000         9
disease_signs_or_symptoms  0.645161290 0.800000000 0.714285714        50
     disease_transmission  0.735294118 0.735294118 0.735294118        34
not_related_or_irrelevant  0.611111111 0.550000000 0.578947368        60
 other_useful_information  0.609929078 0.704918033 0.653992395       122
               prevention  0.729729730 0.586956522 0.650602410        46
                treatment  0.694117647 0.702380952 0.698224852        84

                 accuracy                      0.632882883       444
                macro avg  0.592372417 0.587935156 0.583579374       444
             weighted avg  0.624171748 0.632882883 0.623132963       444



In [18]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
y_pred = rf.predict(X_test)

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [25]:
print(cm)

[[  1   0  15   1   1  12   0   9]
 [  0   4   0   0   0   3   0   2]
 [  0   0  43   0   1   6   0   0]
 [  0   0   1  18   0  15   0   0]
 [  0   0   2   1  29  24   2   2]
 [  1   0   0   0   1 110   1   9]
 [  1   0   4   1   0  11  28   1]
 [  2   0   4   0   1  14   0  63]]


In [26]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, digits=9))

                           precision    recall  f1-score   support

          affected_people  0.200000000 0.025641026 0.045454545        39
           deaths_reports  1.000000000 0.444444444 0.615384615         9
disease_signs_or_symptoms  0.623188406 0.860000000 0.722689076        50
     disease_transmission  0.857142857 0.529411765 0.654545455        34
not_related_or_irrelevant  0.878787879 0.483333333 0.623655914        60
 other_useful_information  0.564102564 0.901639344 0.694006309       122
               prevention  0.903225806 0.608695652 0.727272727        46
                treatment  0.732558140 0.750000000 0.741176471        84

                 accuracy                      0.666666667       444
                macro avg  0.719875706 0.575395696 0.603023139       444
             weighted avg  0.679579581 0.666666667 0.638517301       444

